This notebook contains data generation procedures, which are further used to train a neural network quantum error mitigation.
In particular, here we generate noisy and noise-free vectors of spin magnetizations for echo evolution and forward-in-time evolution.

In [101]:
import numpy as np
import pickle
import os
from datetime import date
import functions
import importlib

importlib.reload(functions)

<module 'functions' from '/home/VNIIA/dvbabukhin/Загрузки/paper_code/functions.py'>

### Echo-evolution data generation

In [95]:
importlib.reload(functions)

"""
data['data'].shape = (number of initial states, number of time points, noise-free(0) or noisy(1), number of spins)
"""

h = 1.0
J = h/2
sample_size = 2400

noise_dict = {
    "depolarizing": [0.011],
}
noise_model = functions.generate_noise_model(noise_dict)
num_qubits = 6
num_trotters = 10
J_array = [J, J, J, J, J, J, J]
h_array = [h, h, h, h, h, h]
time = np.pi
time_points = 5
p_th = 0.8
coupling_map=[[0,1], [0,2], [1,3], [3,2], [2,4], [3,5], [4,5]]

data = functions.generate_data_TFI_echo_dynamics(
    sample_size=sample_size,
    num_qubits=num_qubits, 
    num_trotters=num_trotters,
    time=time,
    time_points=time_points, # should be 2 or more time points
    h_array=h_array,
    J_array=J_array,
    coupling_map=coupling_map,
    p_th=p_th,
    noise_model=noise_model,
    noise_dict=noise_dict,
    periodic_save=True)

no existing data loaded
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350


In [96]:
data['data'].shape

(2401, 5, 2, 6)

In [97]:
importlib.reload(functions)

now = date.today().isoformat()
cwd = os.getcwd()
new_dir = cwd + '/' + now

if not os.path.exists(new_dir):
    os.makedirs(new_dir)
else:
    pass

noise_params = [v for v in noise_dict.values()]
noise_params_ = []
for p in noise_params:
    for par in p:
        noise_params_.append(par)
                
file_name = (data["parameters"]["backend"].name() + "_"
             + "_".join(noise_dict.keys()) + "_" + "_".join(map(str, noise_params_)) 
             + "_%d_qubits"%num_qubits + "_%d_trotters"%data["parameters"]["num of trotters"]
            + "_%d_timesteps"%data['data'].shape[1])
output = open(new_dir + '/{}.pkl'.format(file_name), 'wb')

pickle.dump(data, output)
output.close()

### Forward-in-time-evolution data generation

In [119]:
h = 1.0
J = h/2

noise_dict = {
    "depolarizing": [0.012],
}
noise_model = functions.generate_noise_model(noise_dict)
num_qubits = 6
coupling_map=[[0,1], [0,2], [1,3], [3,2], [2,4], [3,5], [4,5]]
J_array = [J, J, J, J, J, J, J]
h_array = [h, h, h, h, h, h]

data_forward = functions.generate_data_TFI_forward_dynamics(
    sample_size=100,
    num_qubits=num_qubits, 
    num_trotters=10,
    exact_multiplier=10,
    time=2*np.pi,
    time_points=20,
    h_array=h_array,
    J_array=J_array,
    coupling_map=coupling_map,
    p_th=0.8,
    noise_model=noise_model)

In [120]:
data_forward['data'].shape

(100, 20, 3, 6)

In [121]:
from datetime import date

now = date.today().isoformat()
cwd = os.getcwd()
new_dir = cwd + '/' + now

if not os.path.exists(new_dir):
    os.makedirs(new_dir)
else:
    pass

noise_params = [v for v in noise_dict.values()]
noise_params_ = []
for p in noise_params:
    for par in p:
        noise_params_.append(par)
        
file_name = ("test_" + data_forward["parameters"]["backend"].name() + "_"
             + "_".join(noise_dict.keys()) + "_" + "_".join(map(str, noise_params_)) 
             + "_%d_qubits"%num_qubits + "_%d_trotters"%data_forward["parameters"]["num of trotters"])
output = open(new_dir + '/{}.pkl'.format(file_name), 'wb')

pickle.dump(data_forward, output)
output.close()

In [122]:
# Check data

cwd = os.getcwd()
new_dir = cwd + '/data_q_012_J_half_h'
data_name = '/test_qasm_simulator_depolarizing_0.012_6_qubits_10_trotters'
#data_name = '/qasm_simulator_depolarizing_0.012_6_qubits_10_trotters_5_timesteps'

with open(new_dir + data_name + '.pkl', 'rb') as f:
    test_data_loaded = pickle.load(f)
    
test_data_loaded['data'].shape

(10, 20, 3, 6)

#### misc

In [114]:
import os

cwd = os.getcwd()

new_dir = cwd + '/data_q_011_J_half_h'

data_name = '/train_qasm_simulator'

# with open(new_dir + data_name + '.npy', 'rb') as f:
#     train_data_loaded = pickle.load(f)
    
train_data_loaded = np.load(new_dir + '/' + data_name + '.npy')
print(train_data_loaded.shape)

ind = [i if np.sum(v) != 0 else None for i, v in enumerate(train_data_loaded)]
while None in ind:
    ind.remove(None)
    
train_data_loaded = train_data_loaded[ind]
print(train_data_loaded.shape)

(4801, 5, 2, 6)
(2401, 5, 2, 6)


In [115]:
h = 1.0
J = h/2
sample_size = 1

noise_dict = {
    "depolarizing": [0.013],
}
noise_model = functions.generate_noise_model(noise_dict)
num_qubits = 6
num_trotters = 10
J_array = [J, J, J, J, J, J, J]
h_array = [h, h, h, h, h, h]
time = np.pi
time_points = 5
p_th = 0.8
coupling_map=[[0,1], [0,2], [1,3], [3,2], [2,4], [3,5], [4,5]]

data = functions.generate_data_TFI_echo_dynamics(
    sample_size=sample_size,
    num_qubits=num_qubits, 
    num_trotters=num_trotters,
    time=time,
    time_points=time_points, # should be 2 or more time points
    h_array=h_array,
    J_array=J_array,
    coupling_map=coupling_map,
    p_th=p_th,
    noise_model=noise_model,
    noise_dict=noise_dict,
    periodic_save=True)

data_dict = {
    "data": train_data_loaded,
    "parameters":
    {
        "init state": data['parameters']['init state'],
        "num of qubits": num_qubits,
        "num of trotters": num_trotters,
        "total sim time": time,
        "time points": time_points,
        "h values": h_array,
        "J values": J_array,
        "coupling map": coupling_map,
        "p threshold": p_th,
        "circuit": data['parameters']['circuit'],
        "backend": data['parameters']['backend']
    }
}

file_name = f'data_t_{time_points}'
output = open(new_dir + '/{}.pkl'.format(file_name), 'wb')
pickle.dump(data_dict, output)
output.close()

no existing data loaded
0


In [92]:
new_dir = cwd + '/data_q_013_J_half_h'

data_name = '/test_qasm_simulator'

# with open(new_dir + data_name + '.pkl', 'rb') as f:
#     test_data_loaded = pickle.load(f)

test_data_loaded = np.load(new_dir + '/' + data_name + '.npy')

print(test_data_loaded.shape)

ind = [i if np.sum(v) != 0 else None for i, v in enumerate(test_data_loaded)]
while None in ind:
    ind.remove(None)
    
test_data_loaded = test_data_loaded[ind]
print(test_data_loaded.shape)

(100, 20, 3, 6)
(100, 20, 3, 6)


In [94]:
h = 1.0
J = h/2
sample_size = 1

noise_dict = {
    "depolarizing": [0.013],
}
noise_model = functions.generate_noise_model(noise_dict)
num_qubits = 6
num_trotters = 10
J_array = [J, J, J, J, J, J, J]
h_array = [h, h, h, h, h, h]
time = np.pi
time_points = 20
p_th = 0.8
coupling_map=[[0,1], [0,2], [1,3], [3,2], [2,4], [3,5], [4,5]]

data = functions.generate_data_TFI_forward_dynamics(
    sample_size=sample_size,
    num_qubits=num_qubits, 
    num_trotters=num_trotters,
    time=time,
    time_points=time_points, # should be 2 or more time points
    h_array=h_array,
    J_array=J_array,
    coupling_map=coupling_map,
    p_th=p_th,
    noise_model=noise_model)

data_dict = {
    "data": test_data_loaded,
    "parameters":
    {
        "init state": data['parameters']['init state'],
        "num of qubits": num_qubits,
        "num of trotters": num_trotters,
        "total sim time": time,
        "time points": time_points,
        "h values": h_array,
        "J values": J_array,
        "coupling map": coupling_map,
        "p threshold": p_th,
        "circuit": data['parameters']['circuit'],
        "backend": data['parameters']['backend']
    }
}

file_name = f'data_t_{time_points}'
output = open(new_dir + '/{}.pkl'.format(file_name), 'wb')
pickle.dump(data_dict, output)
output.close()